<a target="_blank" href="https://colab.research.google.com/github/amlalejini/alife-2024-phylo-tutorial/blob/main/notebooks/phylotrackpy-async-gen-example-completed.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Phylotrackpy with asynchronous generations example (completed)

This notebook contains a very simple evolving system with asynchronous generations 
wherein organisms reproduce as they accumulate the requisite resources.
Each organism's genome is a binary string, and organism's accumulate resources as 
a function of the proportion of 1s in their genome. 
I.e., this is a one-max environment!  

The example code in this notebook incorporates phylogeny tracking.
Below, we link to some existing examples and the `phylotrackpy` documentation to 
for your reference. 
If you're working on this during the tutorial at ALife 2024, feel free to ask for our help in understanding the code! 
We're happy to walk you through anything!  

## Helpful reference material

- `phylotrackpy` documentation: <https://phylotrackpy.readthedocs.io/en/latest/introduction.html>
- `phylotrackpy` GitHub repository: <https://github.com/emilydolson/phylotrackpy>
- Example code from an ALife 2023 tutorial: <https://github.com/emilydolson/alife-phylogeny-tutorial/blob/main/perfect_tracking_final.ipynb> 
  - Scroll down to the `phylotrackpy` heading!

## Setup

First, install required Python packages for this example (e.g., phylotrackpy).

### Local setup

If you are running this locally, we recommend making a Python virtual environment to keep your local Python installation clean: 

```
python -m venv venv
source venv/bin/activate
!python -m pip install -r requirements.txt
```

### Google Colab setup

If you're running this on Google Colab, you can run the following python cell to install the requisite packages!

In [ ]:
!python -m pip install phylotrackpy
!python -m pip install polars       # Used by example for data tracking

In [ ]:
from phylotrackpy import systematics
import random
import polars as pl

# Seed random number generator
random.seed(8)

## System implementation

The `Organism` class (below) defines organisms with a binary genome, resource count, and taxon id.
The taxon id member keeps track of the organism's taxon in the phylogeny (which is helpful for `phylotrackpy`'s phylogeny tracker).

In [ ]:
class Organism:
    def __init__(
        self,
        num_genes:int = 10,
        randomize_genome:bool = False,
        init_resources:float = 0.0
    ):
        # Genomes are vectors of binary values
        self.genome = [bool(random.randint(0, 1)) if randomize_genome else False for _ in range(num_genes)]
        # Organisms have resources that determine whether they can reproduce
        self.resources = init_resources
        # Organisms keep track of their taxon id (useful for phylogeny tracking)
        self.taxon_id = None

    @classmethod
    def FromGenome(cls, genome:list):
        # Create new organism from a given genome.
        org = cls(
            num_genes = 0,
            randomize_genome = False,
            init_resources = 0.0
        )
        org.SetGenome(genome)
        return org

    def GetGenome(self):
        return self.genome

    def SetGenome(self, genome:list):
        self.genome = [gene for gene in genome]

    def GetResources(self):
        return self.resources

    def SetResources(self, amount:float):
        self.resources = amount

    def IncResources(self, amount:float=1.0):
        self.resources += amount

    def DecResources(self, amount:float=1.0):
        self.resources -= amount

    def GetTaxonID(self):
        return self.taxon_id

    def SetTaxonID(self, id):
        self.taxon_id = id

    def Mutate(self, per_site_mut_rate:float=0.01):
        num_muts = 0
        for gene_i in range(len(self.genome)):
            if (random.random() <= per_site_mut_rate):
                self.genome[gene_i] = not self.genome[gene_i]
                num_muts += 1
        return num_muts

The "world" implementation is below. Organisms exist in a "well-mixed" environment (the `world` list).
On each update, an organism is chosen at random to "execute", gaining resources proportional to the number of 1s in its genome. 
That organism then reproduces if it has sufficient resources. 

In [ ]:
# World parameters
max_world_size = 500           # Determines maximum number of organisms in world
updates = 20000                 # How long to run system?
repro_cost = 10.0              # Cost of reproduction
max_resource_intake = 5        # Maximum amount of resources an organism can gain when "executed"
per_site_mut_rate = 0.05       # Per-site mutation rate for offspring
genome_length = 100            # Length of organism genomes
print_resolution = 100         # Print world status at this interval
snapshot_resolution = 2000     # Snapshot the phylogeny at this interval
summary_output_resolution = 10 # Interval to save summary information

assert(max_world_size > 0)
assert(updates > 0)
assert(per_site_mut_rate >= 0 and per_site_mut_rate <= 1.0)
assert(genome_length > 0)

# Create a new Systematics object to track the population's phylogeny
# - Taxa in the phylogeny will be defined by genotypes (i.e., offspring with the same genotype as their parent will belong to the same taxon).
phylo_tracker = systematics.Systematics(
    lambda org: org.GetGenome(),
    store_pos = False
)
# Configure snapshots to output basic information about a taxon
phylo_tracker.add_snapshot_fun(
    lambda tax: str(list(map(int,tax.get_info()))).replace(",", " "),
    "genome",
    "Taxon's genome"
)
phylo_tracker.add_snapshot_fun(
    lambda tax: str(sum(tax.get_info())),
    "fitness",
    "Number of ones in taxon genome"
)
# Initialize phylogeny tracker's update to 0
phylo_tracker.set_update(0)

# Create a list to hold phylo metrics over time
data = []

# Create population seeded with initial organism
world = [Organism(num_genes=genome_length, randomize_genome=False)]
# Add initial organism to phylogeny tracker
world[0].taxon_id = phylo_tracker.add_org(world[0])

# Execute world for configured number of updates
for update in range(0, updates+1):
    if (update % print_resolution) == 0:
        print(f"---Update {update}---")
        print(f"  Population size={len(world)}")
        print(f"  Max fitness={max([sum(world[i].GetGenome()) for i in range(len(world))])}")

    # Set phylo tracker's update to current world update
    phylo_tracker.set_update(update)

    # Get ID of individual to "execute"
    pop_id = random.randrange(0, len(world)) if len(world) > 1 else 0
    cur_org = world[pop_id]

    # Calculate resource gain
    fit = sum(cur_org.GetGenome())
    resource_gain = max((fit / genome_length) * max_resource_intake, 0.1)
    cur_org.IncResources(resource_gain)

    # Check if organism has resources to reproduce
    if cur_org.GetResources() >= repro_cost:
        # Current organism pays cost of reproduction
        cur_org.DecResources(repro_cost)
        # Create offspring as copy of current organism
        offspring = Organism.FromGenome(cur_org.GetGenome())
        mut_count = offspring.Mutate(per_site_mut_rate)
        # Add offspring to phylogeny tracker, note its taxon ID
        offspring_taxon_id = phylo_tracker.add_org(offspring, cur_org.taxon_id)
        offspring.SetTaxonID(offspring_taxon_id)
        # Place offspring into world
        if len(world) < max_world_size:
            # If world not full, just append.
            world.append(offspring)
        else:
            # Otherwise, random replacement.
            offspring_loc = random.randint(0, len(world)-1)
            # Mark organism to be removed from phylogeny for deletion
            phylo_tracker.remove_org(world[offspring_loc].GetTaxonID())
            # Offspring replaces organism at chosen location.
            world[offspring_loc] = offspring

    # Snapshot phylogeny
    if (update % snapshot_resolution) == 0:
        phylo_tracker.snapshot(f"phylo_snapshot_{update}.csv")

    # Record data for this update
    if (update % summary_output_resolution) == 0:
        data.append({
            "update": update,
            "pop_size": len(world),
            "phylo_num_taxa": phylo_tracker.get_num_taxa(),
            "phylo_sum_pairwise_dist": phylo_tracker.get_sum_pairwise_distance(False),
            "phylo_variance_pairwise_dist": phylo_tracker.get_variance_pairwise_distance(False),
            "phylo_phylogenetic_diversity": phylo_tracker.get_phylogenetic_diversity(),
            "phylo_mrca_depth": phylo_tracker.mrca_depth()
        })

# Optional: These phylogenies are a little big to visualize! Visualizing large
# phylogenies is an area of active research. For now, a good strategy is to
# sample the leaf nodes. The code below does that
leaves_to_retain = 10
taxa_to_remove = random.sample(phylo_tracker.get_active_taxa(), phylo_tracker.get_num_active() - leaves_to_retain)
for t in taxa_to_remove:
    for i in range(t.get_num_orgs()):
        phylo_tracker.remove_org(t)


phylo_tracker.snapshot("phylo_snapshot_final.csv")
summary_df = pl.DataFrame(data)
summary_df.write_csv("phylo_summary.csv")

# Visualization

Phylogeny visualization is an area of active research, particularly as it applies to the more complex phylogenies that we often get in ALife research. Here are some suggestions.

## ALife Phylogeny Visualizer

https://emilydolson.github.io/lineage_viz_tool/phylogeny_visualizations/phylogeny.html

This tool is built for visualizing artificial life phylogenies. It is written in Javascript. It expects data in [ALife standard phylogeny data format](https://alife-data-standards.github.io/alife-data-standards/phylogeny). You can upload a file using the file selector, and it should immediately visualize your phylogeny. From there, there are a few settings that you can adjust:

- Scale exponent: Under some evolutionary regimes, most of the interesting phylogenetic structure happens very near the end of evolutionary times. In other cases, it happens early on. To accommodate these differences, the visualizer uses an exponentially-scaled time axis. To adjust the exponent used (and therefore the temporal part of the phylogeny being emphasized), slide the slider.

- Show intermediate taxa: In biology, usually only the tips (i.e. leaf nodes) of the phylogeny are known. In ALife, we usually know details about every node of the phylogeny. We can take advantage of this information. By toggling on this checkbox, the visualizer will display rectangles alongside the phylogeny depicting the portion of evolutionary time that each taxon was alive for. Sometimes, the lineage that lead to an extant taxon will contain many taxa that coexisted with each other. In these cases, rectangles are drawn nexxt to each other.

- You can also adjust the colors of the lines and circles in the phylogeny.

## IcyTree

https://icytree.org/

This is a visualizer built for bioinformaticians. Consequently, it lacks the ALife-specific features. However, it is more mature and contains lots of useful features. To use it, you can use the alife-phyloinformatics-convert python package to convert an ALife Standard Phylogeny file to a bioinformatics standard such as newick. Note that newick format does not always play nicely with very large phylogenies, so you may need to sub-sample yours to get a good visualization.

The following code will convert an ALife standard phylogeny to newick

In [ ]:
!python -m pip install alifedata_phyloinformatics_convert

In [17]:
import alifedata_phyloinformatics_convert as apc

# Optional: if your data isn't already in a phylotrackpy systematics 
# object, load it in
# (only do this with an empty systematics pbject)
#
# phylo_tracker.load_from_file("phylo_snapshot_final.csv")


converter = apc.RosettaTree(phylo_tracker)
newick_string = converter.to_newick()  # returns newick string
with open("phylogeny.nwk", "w") as f:
    f.write(newick_string)

You can then upload this newick file to IcyTree!